In [ ]:
import json
import pandas as pd
import function as f
import pyodbc
from datetime import datetime

In [ ]:
try:
    conexao, cursor = f.Conexion(database="MarvelAPI")
    print("Conexão bem-sucedida!")
except pyodbc.Error as e:
    print(f"Erro ao conectar: {e}")

In [ ]:

def extrair_nomes_descricoes(arquivo_path):
    with open(arquivo_path, 'r', encoding='utf-8') as f:
        dados = json.load(f)
    
    for item in dados:
        if isinstance(item.get('data'), list):
            personagens = item['data']
            break

    nomes_descricoes = []
    for personagem in personagens:
        nomes_descricoes.append({
            'id': personagem.get('id'),
            'nome': personagem.get('name'),
            'descricao': personagem.get('description', 'Sem descrição')
        })
    
    c = 0
    while c < len(nomes_descricoes):
        id = (nomes_descricoes)[c]['id']
        heros = (nomes_descricoes)[c]['nome']
        descricao = (nomes_descricoes)[c]['descricao']
        data = datetime.now()
        try:
            cursor.execute(f'''
                       INSERT INTO marvel.Heroes (id, nome, descricao, data_download)
                       VALUES
                       (?, ?, ?, ?)
                       ''', id, heros, descricao, data)
        except pyodbc.Error as e:
            print(f"Erro = {e}")
        c += 1
        #print(f"Nome = {heros}\nDescrição = {descricao}\nID = {id}\n")
    
    df = pd.DataFrame(nomes_descricoes)
    return df

df_simples = extrair_nomes_descricoes(r'C:\Users\KallebeDanielGuedesA\Documents\MeusProjetosGit\MarvelAPI\data\characters2.json')

In [ ]:
conexao.commit()
conexao.close()
cursor.commit()
conexao.close()